In [1]:
import pandas as pd
import pandas as df
import numpy
import math
import datetime
from datetime import datetime
from datetime import date
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from datetime import datetime as dt
from datetime import datetime, timedelta
import numpy as np
import pickle
import pickle_compat
import csv

from sklearn.neural_network import MLPClassifier

pickle_compat.patch()

In [2]:
def readCSV(cgm, insulin):
    
    filename1 = open(cgm,'r')
    filename2 = open(insulin,'r')
    date_format_str = '%d/%m/%Y %H:%M:%S'

    
    CGMData = []
    
    for col in csv.DictReader(filename1):
        row = []
        row.append(col['Date'])
        row.append(col['Time'])
        row.append(col['Sensor Glucose (mg/dL)'])
        row.append(col['ISIG Value'])
        CGMData.append(row)
        
    CGMData = CGMData[::-1]
    
    CGMDataClean = []
    for i in CGMData:
        arr = []
        month = int(str(i[0]).split('/',3)[0])
        day = int(str(i[0]).split('/',3)[1])
        year = int(str(i[0]).split('/',3)[2])
        hour = int(str(i[1]).split(':',3)[0])
        minute = int(str(i[1]).split(':',3)[1])
        second = int(str(i[1]).split(':',3)[2])
        date = str(day) + '/'+ str(month) +'/'+ str(year) + ' ' +str(hour)+':'+ str(minute)+':'+ str(second)
        dateAndTime = datetime.strptime(date, date_format_str)
        
        arr.append(dateAndTime)
                
        if i[2].strip() and i[3].strip():
            arr.append(float(i[2]))
            arr.append(float(i[3]))
            CGMDataClean.append(arr)
            
    ##############################################
    glucose = []
    isgm = []
    dates = []
    for i in CGMDataClean:
        glucose.append(i[1])
        isgm.append(i[2])
        dates.append(i[0])
        
    cgm = np.array(glucose)
    isg = np.array(isgm)
    cgm1 = (cgm - np.mean(cgm, axis=0))/np.std(cgm,axis =0)
    isg1 = (isg - np.mean(isg, axis=0))/np.std(isg,axis =0)
    
    CGMDataClean1 = []
    for i in range(0, len(CGMDataClean),1):
        arr = []
        arr.append(dates[i])
        arr.append(cgm1[i])
        arr.append(isg1[i])
        CGMDataClean1.append(arr)
        
    
    
    
    ###############################################
    
    
    InsulinData = []
    
    for col in csv.DictReader(filename2):
        row = []
        row.append(col['Date'])
        row.append(col['Time'])
        row.append(col['BWZ Carb Input (grams)'])
        InsulinData.append(row)
        
    InsulinData = InsulinData[::-1]
    
    
    InsulinDataClean = []
    
    for i in InsulinData:
        row = []
        row2 = []
        date = []
        time = []
        
        month = int(str(i[0]).split('/',3)[0])
        day = int(str(i[0]).split('/',3)[1])
        year = int(str(i[0]).split('/',3)[2])
        hour = int(str(i[1]).split(':',3)[0])
        minute = int(str(i[1]).split(':',3)[1])
        second = int(str(i[1]).split(':',3)[2])
        #[[[2017, 9, 5], [13, 14, 52]], 38.0]
        date.append(year)
        date.append(month)
        date.append(day)
        
        time.append(hour)
        time.append(minute)
        time.append(second)
#         print(col,time)

 
        row.append(date)
        row.append(time)
        row2.append(row)
        
        if i[2].strip():
            
            if float(i[2]) > 0 :
                row2.append(float(i[2]))
                
                InsulinDataClean.append(row2)
        
            
            
    return InsulinDataClean, CGMDataClean

In [3]:
def formatDateTimeCGM(CGMData):
    
    formattedArray = []
    
    for i in CGMData:
            
        upperYear = i[0][0][0]
        upperMonth = i[0][0][1]
        upperDay = i[0][0][2]

        upperHour = i[0][1][0]
        upperMinute = i[0][1][1]
        upperSecond = i[0][1][2]
        
        date_2 = str(upperDay) + '/'+ str(upperMonth) +'/'+ str(upperYear) + ' ' +str(upperHour)+':'+ str(upperMinute)+':'+ str(upperSecond)
        date_format_str = '%d/%m/%Y %H:%M:%S'
        start = datetime.strptime(date_2, date_format_str)
        formattedArray.append([start,i[1],i[2]])
        

    return formattedArray

In [4]:
def ReadXlsx(CGM,Insulin):
    
    #creating the matrix with labels below
    filename1 = pd.read_excel(CGM)
    filename2 = pd.read_excel(Insulin)
    #print(filename2.Index[0])
    CGMData_pre = []
    InsulinData_pre = []

    eatingBehavior = filename1['Sensor Glucose (mg/dL)'] #.to_numpy()
    ISIG = filename1['ISIG Value'] #.to_numpy()
    CGMDate = filename1['Date'] #.to_numpy()
    CGMTime = filename1['Time'] #.to_numpy()


    Eats =  filename2['BWZ Carb Input (grams)'] #.to_numpy()
    InsulinDate = filename2['Date'] #.to_numpy()
    InsulinTime = filename2['Time'] #.to_numpy()
    
    InsulinTimeAfter = []
    for i in InsulinTime:
        arr = []
        arr.append(int(str(i).split(':',3)[0]))
        arr.append(int(str(i).split(':',3)[1]))
        arr.append(int(str(i).split(':',3)[2]))
        InsulinTimeAfter.append(arr)
    CGMTimeAfter = []
    for i in CGMTime:
        arr = []
        arr.append(int(str(i).split(':',3)[0]))
        arr.append(int(str(i).split(':',3)[1]))
        arr.append(int(str(i).split(':',3)[2]))
        CGMTimeAfter.append(arr)


    InsulinDateAfter = []
    for i in InsulinDate:
        arr = []
        var = pd.to_datetime(i)
        q= str(var).split()
        arr.append(int(q[0].split('-',3)[0]))
        arr.append(int(q[0].split('-',3)[1]))
        arr.append(int(q[0].split('-',3)[2]))
        InsulinDateAfter.append(arr)


    CGMDateAfter = []
    for i in CGMDate:
        arr = []
        var = pd.to_datetime(i)
        q= str(var).split()
        arr.append(int(q[0].split('-',3)[0]))
        arr.append(int(q[0].split('-',3)[1]))
        arr.append(int(q[0].split('-',3)[2]))
        CGMDateAfter.append(arr)
    
    
    DateandTime = []
    for i in range(0, len(InsulinDateAfter),1):
        arr = []
        arr.append(InsulinDateAfter[i])
        arr.append(InsulinTimeAfter[i])
        DateandTime.append(arr)


    CGMDateandTime = []
    for i in range(0, len(CGMDateAfter),1):
        arr = []
        arr.append(CGMDateAfter[i])
        arr.append(CGMTimeAfter[i])
        CGMDateandTime.append(arr)
    
  
    
    DateAndFood = []
    for i in range(0, len(DateandTime),1):
        arr = []
        arr.append(DateandTime[i])
        arr.append(Eats[i])
        DateAndFood.append(arr)

    DateAndFood = DateAndFood[::-1]


    CGMDateAndFood = []
    for i in range(0, len(CGMDateandTime),1):
        arr = []
        arr.append(CGMDateandTime[i])
        arr.append(eatingBehavior[i])
        arr.append(ISIG[i])
        CGMDateAndFood.append(arr)

    CGMDateAndFood = CGMDateAndFood[::-1]
    
    DataClean = []
    for i in DateAndFood:
        arr = []
        empty = math.isnan(i[1])
        if i[1] > 0 and empty == False:
            arr.append(i[0])
            arr.append(i[1])  
            DataClean.append(arr)
            
            
    CGMDateAndFood = formatDateTimeCGM(CGMDateAndFood)
            
    return DataClean, CGMDateAndFood


In [5]:
def calculatesSlopes(i):

    array = []

    if len(i[1]) >= 3:

            numbers = np.array(i[1])
            #n = 15
                    
            for j in range(0,len(i[1]),2):
                            
                    if j + 2 < len(i[1]):
                        
                        
                        diff_in_hours = i[0][j+2] - i[0][j]
                        diff_in_hours = diff_in_hours.total_seconds() / 3600
                        slope3 = (i[1][j] +i[1][j+2] - 2*i[1][j+1]) / diff_in_hours
                        array.append(slope3)
                        
                                
    return array

In [6]:
def Features(featureSpace):

    
    featuresForEachMeal = []
    for i in featureSpace:
        
        
        array = []
        #print(max(i[1]) - min(i[1]))
        array.append( max(i[1]) - min(i[1]))
        indexMax = i[1].index(max(i[1]))
        indexMin = i[1].index(min(i[1]))
        ################################
        array.append(max(i[1]))
        array.append(indexMax)
        array.append(min(i[1]))
        array.append(indexMin)

        ################################
        higher = i[0][indexMax]
        lower = i[0][indexMin]
        
        diff = higher -lower
        diff_in_hours = diff.total_seconds() / 3600

        #print(i[0][index])
        array.append(round(diff_in_hours,3))
        
        signal = np.array(i[1], dtype=float)
        fourier = np.fft.fft(signal)
        n = signal.size
        sampleRate = 100
        timestep = 0.1/sampleRate
        freq = np.fft.fftfreq(n, d=timestep)
        
    
        f = np.array(i[1], dtype=float)
        x = np.arange(f.size)
        
        x1 = []
        count = 0.083
        for h in f:
            x1.append(count)
            count += 0.083
        
        x1 = np.array(x1)
        #print(x1.size, f.size)
        #break
        #print(modifiedSlope.size)
        #print(x1.size)
        #print(f.size)
        
        gradients = []
        
        
        #################
        
        ###################
        
        if len(freq) >= 4 and len(x) > 4 and x1.size > 4:
            
            array.append(round(freq[1],3))
            array.append(round(freq[2],3))
            array.append(round(freq[3],3))
           
            modifiedSlope = np.gradient(f, x1)
            slope = np.diff(i[1])
            #print(slope)
            
            if len(modifiedSlope) >= 5:
                ################################
                n = 6
                
                #grad = calculatesSlopes(i)
                
               # grad = np.array(grad)                         
                         
                #slopes = (-slope).argsort()[:n]
#                 plt.plot(slopes,slope[slopes],color='green')
                zero_crossings = numpy.where(numpy.diff(np.signbit(modifiedSlope)))[0]
                
                amplitudes = []
                
                for j in zero_crossings:
                    
                    amplitude = abs(modifiedSlope[j])
                    amplitudes.append([j, amplitude])
                        

                sorted_multi_list = sorted(amplitudes, key=lambda x: x[1])
                
               
                if len(amplitudes) >= 5 and len(i[1]) > 0 and len(i[2]) > 0:
                    
                    #print(amplitudes)

                    #plt.plot(range(len(modifiedSlope)),modifiedSlope,label='x',color='red')
                    #plt.plot(zero_crossings,modifiedSlope[zero_crossings],color='blue')

                    #temp = [sorted_multi_list[-5][0],sorted_multi_list[-4][0],sorted_multi_list[-3][0],sorted_multi_list[-2][0],sorted_multi_list[-1][0]]
                    #temp1 = [modifiedSlope[sorted_multi_list[-5][0]],modifiedSlope[sorted_multi_list[-4][0]],modifiedSlope[sorted_multi_list[-3][0]],modifiedSlope[sorted_multi_list[-2][0]],modifiedSlope[sorted_multi_list[-1][0]]]
                    
                   
                    #plt.scatter(temp, temp1,color='black')


        
                    array.append(sorted_multi_list[-1][0])
                    array.append(modifiedSlope[sorted_multi_list[-1][0]])
                    array.append(sorted_multi_list[-2][0])
                    array.append(modifiedSlope[sorted_multi_list[-2][0]])
                    array.append(sorted_multi_list[-3][0])
                    array.append(modifiedSlope[sorted_multi_list[-3][0]])
                    
                    array.append(sorted_multi_list[-4][0])
                    array.append(modifiedSlope[sorted_multi_list[-4][0]])
                    array.append(sorted_multi_list[-5][0])
                    array.append(modifiedSlope[sorted_multi_list[-5][0]])
                    
                    
                    mean = int(np.mean(i[1]))
                    array.append(mean)
                    array.append(int(np.mean(i[2])))
                    array.append(int(np.std(i[1])))
                    array.append(int(np.std(i[2])))

                    if mean > 180:
                        array.append(1)
                    
                    if mean < 70:
                        array.append(-1)
                        
                    if mean >= 70 and mean <= 180:
                        array.append(0)

                    featuresForEachMeal.append(array)


                    #plt.show()
 
 



              
    return featuresForEachMeal

In [8]:
def dataForFeatures(featureSpace, LabelRange):
    
    array = []
    
    for i in LabelRange:
        subarray = []

        lower = i[0]
        upper = i[1]
        
        subarrayTime = []
        subarrayGlucose = []
        subarrayISMG = []
        
        for j in featureSpace:
            
           
            glucose = math.isnan(j[1])
            ismg = math.isnan(j[2])
            
            if j[0] >= lower and j[0] <= upper and glucose == False and ismg == False:
                
                subarrayTime.append(j[0])
                subarrayGlucose.append(j[1])
                subarrayISMG.append(j[2])
        
        if len(subarrayGlucose) > 0 and len(subarrayISMG):
            
            subarray.append(subarrayTime)
            subarray.append(subarrayGlucose)
            subarray.append(subarrayISMG)    
            array.append(subarray)
                
    return array

In [9]:
#DataClean, CGMDateAndFood= ReadXlsx('CGMData670GPatient3.xlsx','InsulinAndMealIntake670GPatient3.xlsx')
DataClean, CGMDateAndFood= readCSV('CGM_patient2.csv','Isulin_patient2.csv')
DataClean2, CGMDateAndFood2 = readCSV( 'CGMData.csv', 'InsulinData.csv')

# CGMDateAndFood = CGMDateAndFood[0:10000]
# CGMDateAndFood2 = CGMDateAndFood2[0:40000]
#print(len(DataClean))
#print(DataClean[0])
#print(CGMDateAndFood[0])
#print(len(CGMDateAndFood))

In [19]:
def extraction(data):

    disregard = [False]
    
    keep = [False]
    mealData = []
    noMealData = []
    
    i = 1
    
    while i < len(data) - 1:
    
            
            #data[i][0]
            #data[i+1][0]
                
            Year = data[i][0][0][0]
            Month = data[i][0][0][1]
            Day = data[i][0][0][2]
            
            Hour = data[i][0][1][0]
            Minute = data[i][0][1][1]
            Second = data[i][0][1][2]
            
        
        
            lowerYear = data[i-1][0][0][0]
            lowerMonth = data[i-1][0][0][1]
            lowerDay = data[i-1][0][0][2]
            
            lowerHour = data[i-1][0][1][0]
            lowerMinute = data[i-1][0][1][1]
            lowerSecond = data[i-1][0][1][2]

            upperYear = data[i+1][0][0][0]
            upperMonth = data[i+1][0][0][1]
            upperDay = data[i+1][0][0][2]

            upperHour = data[i+1][0][1][0]
            upperMinute = data[i+1][0][1][1]
            upperSecond = data[i+1][0][1][2]

            
            date_1 = str(lowerDay) + '/'+ str(lowerMonth) +'/'+ str(lowerYear) + ' ' +str(lowerHour)+':'+ str(lowerMinute)+':'+ str(lowerSecond)
            date_2 = str(upperDay) + '/'+ str(upperMonth) +'/'+ str(upperYear) + ' ' +str(upperHour)+':'+ str(upperMinute)+':'+ str(upperSecond)
            date_3 = str(Day) + '/'+ str(Month) +'/'+ str(Year) + ' ' +str(Hour)+':'+ str(Minute)+':'+ str(Second)

            date_format_str = '%d/%m/%Y %H:%M:%S'

            
            lower = datetime.strptime(date_1, date_format_str)
            upper =   datetime.strptime(date_2, date_format_str)
            current =   datetime.strptime(date_3, date_format_str)

            
            # Get interval between two timstamps as timedelta object
            diffUpper = upper - current
            diffLower = current - lower

            # Get interval between two timstamps in hours
            diffUpper_in_hours = diffUpper.total_seconds() / 3600
            diffLower_in_hours = diffLower.total_seconds() / 3600

                        
#             if diffLower_in_hours < 0.5:
                

            #rule 1
            #might need recursion
            if diffLower_in_hours > 0.5 and diffUpper_in_hours < 2:
                lowerTime = upper + timedelta(hours=0) - timedelta(hours=0.5)
                upperTime = upper + timedelta(hours=2)
                mealData.append([lowerTime,upperTime,data[i][1]])
                i = i + 1

            
            if diffLower_in_hours > 0.5 and diffUpper_in_hours >= 2:
                lowerTime = current - timedelta(hours=0.5)
                upperTime = current + timedelta(hours=2)
                mealData.append([lowerTime,upperTime,data[i][1]])
                

            if diffLower_in_hours > 0.5 and diffUpper_in_hours >= 4:
                lowerTime = current + timedelta(hours=2)
                upperTime = current + timedelta(hours=4)
                noMealData.append([lowerTime,upperTime,data[i][1]])
                
                
            if diffLower_in_hours > 0.5 and diffUpper_in_hours >= 6:
                lowerTime = current + timedelta(hours=4)
                upperTime = current + timedelta(hours=6)
                noMealData.append([lowerTime,upperTime,data[i][1]]) 
               
            
            if diffLower_in_hours > 0.5 and diffUpper_in_hours >= 8:
                lowerTime = current + timedelta(hours=6)
                upperTime = current + timedelta(hours=8)
                noMealData.append([lowerTime,upperTime,data[i][1]])
            
                
            
            #rule 2
            if diffLower_in_hours < 0.5:
                mealData = mealData[:-1]
                

            
            if diffUpper_in_hours >= 2 and diffUpper_in_hours <= 2.04:
                lowerTime = current + timedelta(hours=0) - timedelta(hours=0.5)
                upperTime = current + timedelta(hours=2)
                mealData.append([lowerTime,upperTime,data[i][1]])
                
                lowerTime = upper + timedelta(hours=0) - timedelta(hours=0.5)
                upperTime = upper + timedelta(hours=2)
                mealData.append([lowerTime,upperTime,data[i][1]])
                
                i = i + 1
#                 noMealData.append(data[i])
            
            
            i = i+1



            
    
    return mealData, noMealData

In [20]:
meals, noMeals = extraction(DataClean)
meals2, noMeals2 = extraction(DataClean2)

In [21]:
mealpreFeatureData = dataForFeatures(CGMDateAndFood, meals)
#noMealpreFeatureData = dataForFeatures(CGMDateAndFood, noMeals)

mealpreFeatureData2 = dataForFeatures(CGMDateAndFood2, meals2)
#noMealpreFeatureData2 = dataForFeatures(CGMDateAndFood2, noMeals2)

In [22]:
meal = Features(mealpreFeatureData)
noMeal = Features(noMealpreFeatureData)

meal2 = Features(mealpreFeatureData2)
noMeal2 = Features(noMealpreFeatureData2)


# noMeal = noMeal[1:29]
# noMeal2 = noMeal2[1:29]